# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [17]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Find the employee who earns the highest salary""", context_user))

```sql
SELECT employee_name
FROM employees
WHERE salary = (SELECT MAX(salary) FROM employees);
```


In [11]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Find the employee who earns the highest salary", old_context_user))

This is your SQL:
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 1;
```

This SQL query selects the name and salary of the employee who earns the highest salary by joining the "employees" and "salary" tables on the employee ID. It then orders the results by salary in descending order and limits the output to only the top result, which is the employee with the highest salary.


In [15]:
#new
print(return_CCRMSQL("Show the average salary grouped by year", context_user))

```sql
SELECT year, AVG(salary) AS average_salary
FROM employees
GROUP BY year;
```


In [16]:
#old
print(return_CCRMSQL("Show the average salary grouped by year", old_context_user))

This is your SQL:
```sql
SELECT year, AVG(salary) AS average_salary
FROM salary
GROUP BY year;
```

This SQL query selects the year and calculates the average salary for each year by grouping the data based on the year column in the salary table.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [19]:

# ==========================================================================================
context_user = context.copy()
old_context_user = old_context.copy()

print("### Variation 1 - New Context")
print(return_CCRMSQL("Find all employees who earn more than 60000", context_user))

print("\n### Variation 1 - Old Context")
print(return_CCRMSQL("Find all employees who earn more than 60000", old_context_user))



# ==========================================================================================
context_user = context.copy()
old_context_user = old_context.copy()

print("\n\n### Variation 2 - New Context")
print(return_CCRMSQL("Show the average salary by year", context_user))

print("\n### Variation 2 - Old Context")
print(return_CCRMSQL("Show the average salary by year", old_context_user))


# =============================================================================================
context_user = context.copy()
old_context_user = old_context.copy()

print("\n\n### Variation 3 - New Context")
print(return_CCRMSQL("List the employees with their institution of study", context_user))

print("\n### Variation 3 - Old Context")
print(return_CCRMSQL("List the employees with their institution of study", old_context_user))


### Variation 1 - New Context
```sql
SELECT * 
FROM employees
WHERE salary > 60000;
```

### Variation 1 - Old Context
This is your SQL:
```sql
SELECT employees.name
FROM employees
JOIN salary ON employees.ID_usr = salary.ID_usr
WHERE salary.salary > 60000;
```

This SQL query retrieves the names of employees who earn more than 60000 by joining the "employees" table with the "salary" table on the ID_usr column and filtering the results based on the salary amount.


### Variation 2 - New Context
```sql
SELECT year, AVG(salary) AS average_salary
FROM employees
GROUP BY year;
```

### Variation 2 - Old Context
This is your SQL:
```sql
SELECT year, AVG(salary) AS average_salary
FROM salary
GROUP BY year;
```

This SQL query selects the year and calculates the average salary for each year from the "salary" table. The results are grouped by year using the GROUP BY clause, and the AVG() function is used to calculate the average salary for each year.


### Variation 3 - New Context
```sql
SELE

Report on SQL Query Generation with GPT

In this lab I worked on using GPT to make SQL queries. I tried two different kinds of prompts: the old one, where tables were written in a JSON style, and the new one, where tables were written with CREATE TABLE plus sample data and some example queries. The goal was to see which one worked better.

What I Found

When I asked questions like “Who is the highest paid employee?” or “Which institution has the highest average salary?”, the new prompt usually gave better answers. The SQL was correct, it used the right tables, and it added joins when needed. It also explained the result clearly.

The old prompt sometimes worked but not always. A few times it gave me SQL that was missing things, like not using GROUP BY when it should, or it even made up a column that didn’t exist. That showed me that the old style is less reliable.

What I Learned

I learned that the way I write the prompt really matters. The new style with CREATE TABLE, sample rows, and examples helped GPT a lot. It made the answers more accurate and less random. The old style made GPT more likely to guess or “hallucinate.”

Another thing I learned is that GPT can look very confident even when the answer is wrong. So it’s important to double-check the SQL before using it.

Overall, the main lesson is: clearer prompts = better results. The new method worked way better, and now I understand how important it is to guide the model with the right context.